# 주문집행 시뮬레이션
- 크레프트 태크놀로지스 AXE 팀은 주문집행 문제를 강화학습을 통해 해결합니다.
- 하지만 기존의 toy project와는 다르게, 실제 상황에 강화학습을 적용하는 시도는 생각보다 많은 장애물을 마주하게 됩니다.
- 이 샘플 태스크는 그 응용력을 아주 간단한 범위에서 시험합니다.

**주의 : '완벽하게' 푸는 것이 이 태스크의 목적은 아닙니다. 무엇보다, 자신의 생각을 코드로 구현하려는 시도가 중요합니다.**


In [1]:
# 주문집행 시뮬레이션 모듈 import
from env import OrderSimulation1, OrderSimulation2

In [2]:
# 주문 집행 시뮬레이션 인스턴스입니다.
# 각 reset마다 랜덤으로 (사야하는 주식의 수 / 타임 스텝) 을 설정합니다.
# 목표는, 최소 비용으로 주식을 구매하는 것 입니다.
env = OrderSimulation1()

## 함수 스펙 설명

In [3]:
# reset -> 초기 호가정보 반환[5개의 time frame][매수 3호가 ~ 매도 3호가]
order_books = env.reset()
print("호가창 길이: {}".format(len(order_books)))
order_books[-1]

호가창 길이: 5


464000    102
465000    141
466000    161
467000     86
468000     67
469000      2
dtype: int64

In [4]:
# mission_info -> 현재 episode 의 목표 구매 수량과 maximum time step
env.mission_info()

{'total_step': 23, 'mission_buy': 300}

In [6]:
# step -> action(매수 1호가 주문, 매수 2호가 주문, 매수 3호가 주문, 시장가 주문) 을 받아 시뮬레이션을 처리
# 편의상 step 후 체결되지 않은 주문은 모두 취소함을 가정합니다.
# 반환 : 새로운 호가창 시퀀스(1~4번쨰는 이전 시퀀스의 2~5번쨰 항목과 동일), 이번 기 매수체결 정보, 전체 기 매수체결 정보. passed time(지난 observation 으로부터 경과한 시간, 단위:초)
# 모든 주문은 기존에 호가창에 있는 대기 잔량이 체결된 후에 체결됩니다. 주문이 무조건적으로 체결되는 것이 아닙니다.
# 필요 수량 이상으로 주문하는 경우 에러가 발생합니다.
order_books, executed_this_step, executed_every_step, passed_time = env.step([30, 10, 20, 5])
executed_this_step

469000    5
dtype: int64

In [7]:
# get_left_observation() 은 일종의 치트 메소드 입니다.
# 현재부터 episode의 끝까지 발생할 다음 호가창 상태를 미리 알려줍니다.
# (이 환경은 상태전이와 action의 독립을 가정하고 있습니다)
env.get_left_observation()

[468000    110
 469000     94
 470000     24
 471000    112
 472000     97
 473000     63
 dtype: int64,
 468000     52
 469000    194
 470000    164
 471000     85
 472000    143
 473000     24
 dtype: int64,
 467000     35
 468000    122
 469000     72
 470000    157
 471000     75
 472000     53
 dtype: int64,
 467000     97
 468000    197
 469000    155
 470000     28
 471000    124
 472000      3
 dtype: int64,
 466000      8
 467000      1
 468000     24
 469000    174
 470000     61
 471000    136
 dtype: int64,
 468000     67
 469000     31
 470000    103
 471000      1
 472000    173
 473000     75
 dtype: int64,
 471000     60
 472000    177
 473000     63
 474000    131
 475000     77
 476000    165
 dtype: int64,
 468000     48
 469000    186
 470000    111
 471000    122
 472000    146
 473000      8
 dtype: int64,
 469000     72
 470000     33
 471000     32
 472000    150
 473000    145
 474000    100
 dtype: int64,
 469000    135
 470000     82
 471000      2
 472000   

# Task - 리워드가 없는데요
강화학습의 핵심은 reward입니다. <br/>
하지만 보시다 시피 주문집행에서 reset과 step은 명시적인 reward를 반환하지 않습니다.(사소하지만 done 역시 마찬가지입니다.) <br/>
기존 OrderSimulation class를 이용, 아래 형식에 맞추어 환경을 다시 정의해보십시오.<br/>


*HINT* : 이 simulation은 학습을 위한 gym입니다. <br/>
action을 결정하는 데는 사용해서는 안 되지만, reward를 계산하는 데 미래 데이터를 쓰는데 주저하지 마십시오!

*WARNING* : 정답은 없습니다. 그럴 듯한 학습 목표를 세우는 데 초점을 맞추시면 됩니다.

In [8]:
class OrderEnv:
    
    # TODO
    # return: state, reward, done, info
    def reset(self, action):
        raise NotImplementedError

    # TODO
    # return: state, reward, done, info
    def step(self, action):
        raise NotImplementedError

# Task2 - And then...?
이제 본격적으로 해당 환경을 이용하여 학습 알고리즘을 짜봅시다.
---
*직접 구현하는 건 시간상으로도 불가능하고, Sample Gym 특성 상 실제와 유사한 동작을 보장할 수 없습니다.
따라서, 지원자 분의 사고를 서술하는 것으로 대체합니다*

1. `env.step()` 에서나 오는 마지막 반환값은 passed time, 지난 reset 혹은 step으로부터 경과한 시간을 의미합니다.
  - **step**이 명확한 여타 gym 환경과는 달리, 주문집행 환경은 호가/체결이 발생하는 빈도가 일정하지 않습니다.
  - 예를 들어, 장 시작으로부터 1초, 2초, 3초, 4초, 5초에 호가/체결이 하나씩 들어오는 케이스와 이
  1초, 1시간 1초, 2시간 1초, 2시간 2초, 2시간 3초에 호가/체결 이 하나씩 들어오는 케이스는 observation 만으로는 구분되지 않습니다.
  - 이 것이 기존의 강화학습 예제들과 비교하여 어떤 문제점을 야기할지, 해결방법이 떠오른다면 무엇일지 설명해주세요
  - 만약 해결방법을 제시한다면, 최대한 구체적으로 적어주세요. 특히, 그 해결방법이 다른 새로운 문제를 야기하지는 않는지,
  새로운 문제는 어떻게 해결할 수 있는지 꼬리에 꼬리를 무는 방식으로 적어주시면 더 좋습니다!

2. 문제 상황과 observation, 직접 정의하신 reward를 보았을 때 어떤 알고리즘이 먼저떠오르는지, 그 이유는 무엇인지 자유롭게 서술해주세요!